In [63]:
import torch
import os
from pathlib import Path
import numpy as np
from yt_dlp import YoutubeDL
import glob
from operator import itemgetter
from typing import List

from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.retrievers.multi_query import MultiQueryRetriever

from tqdm.notebook import tqdm
import os

In [2]:
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/huggingface-models'

In [3]:
# loader = PyPDFLoader("/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/data/meditations.pdf")
# documents = loader.load()

### Load and Split

In [3]:
# way faster than PyPDFLoader
loader = PyMuPDFLoader("/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/data/meditations.pdf")
documents = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    # separator="\n", # where to split the text
    chunk_size=1000, # how big each chunk should be 
    chunk_overlap=100,
    length_function=len,
    # add_start_index=True
)
docs = text_splitter.split_documents(documents)

In [5]:
print('Chunks: ', len(docs), 'Pages: ', len(documents))

Chunks:  550 Pages:  128


### Embedding

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-small-en-v1.5', 
    model_kwargs={'device': 'cpu'},
    show_progress=True
)

In [7]:
db = FAISS.load_local('/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/rag-optimizations/faiss_pdf', embeddings, allow_dangerous_deserialization=True)

In [9]:
# db = FAISS.from_documents(docs, embeddings)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

In [10]:
# db.save_local('/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/rag-optimizations/faiss_pdf')

In [8]:
vector_res = db.similarity_search("What is meditation?", k=10)
vector_res

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(id='87e8461d-306f-4ceb-8512-5ed5a385561e', metadata={'producer': 'macOS Version 14.2.1 (Build 23C71) Quartz PDFContext', 'creator': 'Safari', 'creationdate': "D:20240110035838Z00'00'", 'source': '/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/data/meditations.pdf', 'file_path': '/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/data/meditations.pdf', 'total_pages': 128, 'format': 'PDF 1.4', 'title': 'https:/www.maximusveritas.com/wp-content/uploads/2017/09/Marcus-Aurelius-Meditations', 'author': 'Arshad Kazi', 'subject': '', 'keywords': '', 'moddate': "D:20240110035838Z00'00'", 'trapped': '', 'modDate': "D:20240110035838Z00'00'", 'creationDate': "D:20240110035838Z00'00'", 'page': 107}, page_content='XXIV. What doest thou desire? To live long. What? To enjoy the operations of a sensi-\ntive soul; or of the appetitive faculty? or wouldst thou grow, and then decrease \nagain? Wouldst thou long be able to talk, to think and reason with thyself? Which of'),
 Documen

### LLM Baseline Response

In [14]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/huggingface-models/models--Qwen--Qwen2.5-0.5B-Instruct/snapshots/7ae557604adf67be50417f59c2c2f167def9a775', base_url='http://0.0.0.0:8000/v1', api_key='n')

In [9]:
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible.

Context: {context}

Question: {question}

Helpful Answer:"""

In [16]:
rag_res = model.invoke(
    prompt_template.format(
        context=''.join([i.page_content for i in vector_res]), 
        question="What is meditation?"
    )
)
print(rag_res.content)

Meditation refers to the practice of focusing one's mind on a single object or idea, often through mindfulness exercises or philosophical reflection. It involves cultivating a sense of inner calm and concentration, allowing the mind to become aware of its thoughts and emotions without judgment. Meditation helps individuals develop greater awareness, reduce stress, and enhance overall mental clarity and self-awareness.


### Query Rewrite

In [11]:
# Reference: https://arxiv.org/abs/2305.14283
# Remove noise in the original user query using LLM, so LLM can focus on 
# main topic of the query and generate the best answers

In [18]:
query = "What is meditation? Response with example."

distracted_query =  query + " Hahaha!"

rewrite_template = """You are an LLM's assistant, your job is to identify the main theme of a query and remove any noise that could disturb the LLM's answer.
Given the original question, rewrite it so that LLM can generate the best possible answer.

original question: {original_question}

Rewritten question:
"""

rewrite_query = model.invoke(
    rewrite_template.format(
        original_question=distracted_query
    )
)
print(rewrite_query.content)

What is meditation?


#### Impact on retrieval

In [27]:
def format_retrieval(docs):
    for doc in docs:  # or whatever your list variable is called
        page = doc.metadata.get('page', 'N/A')
        print(f"Page: {page}")
        print("Content:")
        print(doc.page_content)
        print("="*40)  # separator for readability

In [29]:
original_res = db.similarity_search(query, k=5)
distracted_res = db.similarity_search(distracted_query, k=5)
rewrite_res = db.similarity_search(rewrite_query.content, k=5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
format_retrieval(original_res)

Page: 69
Content:
appetites, to discern rightly all plausible fancies and imaginations, to contemplate 
the nature of the universe; both it, and things that are done in it. In which kind of 
contemplation three several relations are to be observed The first, to the apparent 
secondary cause. The Second to the first original cause, God, from whom originally 
proceeds whatsoever doth happen in the world. The third and last, to them that we 
live and converse with: what use may be made of it, to their use and benefit. 
XXVI. If pain be an evil, either it is in regard of the body; (and that cannot be, be-
cause the body of itself is altogether insensible): or in regard of the soul But it is in 
the power of the soul, to preserve her own peace and tranquillity, and not to suppose 
that pain is evil. For all judgment and deliberation; all prosecution, or aversation is 
from within, whither the sense of evil (except it be let in by opinion) cannot penetrate.
Page: 88
Content:
there, which whe

In [31]:
format_retrieval(distracted_res)

Page: 88
Content:
there, which when as a mere naturalist, thou hast barely considered of according to 
their nature, thou doest let pass without any further use? Whereas thou shouldst in 
all things so join action and contemplation, that thou mightest both at the same time 
attend all present occasions, to perform everything duly and carefully and yet so in-
tend the contemplative part too, that no part of that delight and pleasure, which the 
contemplative knowledge of everything according to its true nature doth of itself af-
ford, might be lost. Or, that the true and contemplative knowledge of everything ac-
cording to its own nature, might of itself, (action being subject to many lets and im-
pediments) afford unto thee sufficient pleasure and happiness. Not apparent indeed, 
but not concealed. And when shalt thou attain to the happiness of true simplicity, 
and unaffected gravity? When shalt thou rejoice in the certain knowledge of every
Page: 107
Content:
XXIV. What doest thou de

In [32]:
format_retrieval(rewrite_res)

Page: 107
Content:
XXIV. What doest thou desire? To live long. What? To enjoy the operations of a sensi-
tive soul; or of the appetitive faculty? or wouldst thou grow, and then decrease 
again? Wouldst thou long be able to talk, to think and reason with thyself? Which of
Page: 69
Content:
appetites, to discern rightly all plausible fancies and imaginations, to contemplate 
the nature of the universe; both it, and things that are done in it. In which kind of 
contemplation three several relations are to be observed The first, to the apparent 
secondary cause. The Second to the first original cause, God, from whom originally 
proceeds whatsoever doth happen in the world. The third and last, to them that we 
live and converse with: what use may be made of it, to their use and benefit. 
XXVI. If pain be an evil, either it is in regard of the body; (and that cannot be, be-
cause the body of itself is altogether insensible): or in regard of the soul But it is in 
the power of the soul, to pr

#### Impact on generation

In [33]:
rag_res = model.invoke(
    prompt_template.format(
        context=''.join([i.page_content for i in distracted_res]), 
        question=distracted_query
    )
)
print(rag_res.content)

Meditation involves focusing one's attention on a particular object, thought, or activity to achieve a state of calmness, concentration, or spiritual clarity. It can involve various techniques such as mindfulness, breathing exercises, visualization, or guided meditation. Examples include sitting quietly in a quiet place, closing your eyes, counting your breaths, or practicing meditation through journaling or writing down thoughts. Meditation helps reduce stress, improve focus, and enhance mental well-being.


In [34]:
rag_res = model.invoke(
    prompt_template.format(
        context=''.join([i.page_content for i in rewrite_res]), 
        question=rewrite_query
    )
)
print(rag_res.content)

To live long, enjoy the operation of a sensitive soul, and ponder the nature of the universe. Pain is neither inherently evil nor a distraction from living well. Meditation is about focusing on your thoughts and feelings, and understanding their meaning. Realizing the interconnectedness of all things can help you appreciate life more deeply.


### Query Expansion (MultiQueryRetriever)

To use LLM as an agent to expand the original query

![queryexpansion](img/query-expansion-multiquery.png)

Reference: https://python.langchain.com/docs/how_to/MultiQueryRetriever/

In [38]:
# Output parser will split the LLM result into a list of queries
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines

output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [44]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [39]:
# Chain
llm_chain = QUERY_PROMPT | model | output_parser

In [45]:
# Run
retriever = MultiQueryRetriever(
    retriever=db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.invoke(query)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What does meditation involve?', '  2. How do you practice meditation?', '  3. Is there any specific time or place for meditation?', '  4. Can you explain what meditation entails in simple terms?', '  5. Is there a particular type of meditation that you prefer?', 'Alternative Questions:', '6. In what ways can meditation be practiced?', '  7. How often should I practice meditation daily?', '  8. Are there certain types of meditation that work better for me?', '  9. Can meditation help improve my concentration and focus?', '  10. Do you have any tips for beginners who want to start practicing meditation?', 'These alternatives aim to explore various aspects of meditation while still maintaining the essence of the original question.']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [46]:
unique_docs

[Document(id='43a9f86b-ab34-4b01-b41d-aa6226f104e1', metadata={'producer': 'macOS Version 14.2.1 (Build 23C71) Quartz PDFContext', 'creator': 'Safari', 'creationdate': "D:20240110035838Z00'00'", 'source': '/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/data/meditations.pdf', 'file_path': '/Users/wangzeyu/Desktop/Github projects/legalai-chatbot/data/meditations.pdf', 'total_pages': 128, 'format': 'PDF 1.4', 'title': 'https:/www.maximusveritas.com/wp-content/uploads/2017/09/Marcus-Aurelius-Meditations', 'author': 'Arshad Kazi', 'subject': '', 'keywords': '', 'moddate': "D:20240110035838Z00'00'", 'trapped': '', 'modDate': "D:20240110035838Z00'00'", 'creationDate': "D:20240110035838Z00'00'", 'page': 69}, page_content='appetites, to discern rightly all plausible fancies and imaginations, to contemplate \nthe nature of the universe; both it, and things that are done in it. In which kind of \ncontemplation three several relations are to be observed The first, to the apparent \nsecond

In [ ]:
# then ask LLM to answer based on these retrieved docs
res = model.invoke(
    prompt_template.format(
        context=''.join([i.page_content for i in unique_docs]), 
        question=query
    )
)
print(res.content)

### Query Decomposition

Query Decomposition, also known as sub-query generation, is the process of splitting the original input into distinct sub-queries and using the answers, questions, and retrieved documents to build a final response. The idea is to break a big problem into smaller sub-problems and use cumulative reasoning to form the final response.

In [79]:
import re
def res_parser(response):
    # Split on the pattern: newline, digit, dot, space
    items = [item.strip() for item in re.split(r'(?:^|\n)\d+\.\s*', response) if item.strip()]
    return items

In [56]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (2 queries):"""
decomp_template = PromptTemplate.from_template(template)

#### Answer recursively

The retrieved documents and the previous answers from the sub-query are used to answer the current sub-query.

![recursiveanswer](img/decomp-recursive-answer.png)

In [81]:
# decomp chain
decomp_chain = (decomp_template | model)

# Run
questions = [q for q in res_parser(decomp_chain.invoke({"question": query}).content) if q != '']
print('Main Query: ',query)
print('\nSub-queries:')
questions

Main Query:  What is meditation? Response with example.

Sub-queries:


['What does meditation primarily focus on?\n   - Example response: Meditation often involves deep relaxation and focused awareness.',
 'How does mindfulness contribute to the practice of meditation?\n   - Example response: Mindfulness helps practitioners maintain attention on their present moment experiences without judgment or distraction.']

In [60]:
# LLM Response Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}

<<ANSWER>>Answer:\n
"""
response_prompt = PromptTemplate.from_template(template)

In [61]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""
    answer = answer.split('<<ANSWER>>Answer:')[-1].strip('\n')
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

In [82]:
q_a_pairs = ""
for i, q in enumerate(questions):
    
    # reference to building chain: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
    rag_chain = (
        {
            "context": itemgetter("question") | db.as_retriever(),
            "question": itemgetter("question"),
            "q_a_pairs": itemgetter("q_a_pairs")
        }
        | response_prompt
        | model
        # | output_parser
    )
    
    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q, answer.content)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

print('\nFinal Answer:\n' +  answer.content.split('<<ANSWER>>Answer:')[-1].strip('\n'))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Final Answer:
Mindfulness contributes to the practice of meditation through several key aspects:

1. **Awareness and Presence**: Mindfulness encourages practitioners to pay attention fully and deeply on their present experience without getting distracted by external stimuli or past events.

2. **Non-Judgmental Awareness**: By focusing solely on the present moment, practitioners cultivate non-judgmental awareness. This leads to greater acceptance and peace of mind, allowing for easier access to deeper levels of awareness.

3. **Detachment from Distractions**: Practitioners learn to detach from distractions and interruptions during meditation, fostering a clearer focus on their internal state and experiences.

4. **Self-Compassion**: Mindfulness practices help practitioners develop self-compassion, recognizing that each moment brings challenges but also opportunities for growth and development.

5. **Emotional Regulation**: Through regular mindfulness exercises, individuals can improve 

#### Answer individually

![individualanswer](img/decomp-individual-answer.png)

In [93]:
def split_and_answer(question, prompt, expansion_chain):
    """RAG on each sub-question"""

    # query decomposition for more sub-questions
    res = expansion_chain.invoke({"question": question})
    sub_questions = [q for q in res_parser(res.content) if q != '']

    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in tqdm(sub_questions):
        
        # Retrieve documents for each sub-question
        retrieved_docs = db.similarity_search(sub_question, k=3)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt | model).invoke({"context": ''.join([i.page_content for i in retrieved_docs]), "question": sub_question})
        rag_results.append(answer)
    
    return rag_results, sub_questions

answers, questions = split_and_answer(query, PromptTemplate.from_template(prompt_template), decomp_chain)

  0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Prompt
final_template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}

<<ANSWER>>Answer:\n
"""

final_template  = PromptTemplate.from_template(final_template)

# put all qa pairs together as a context
formatted_string = ""
for i, (question, answer) in enumerate(zip(questions, answers), start=1):
    formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
formatted_string = formatted_string.strip()

# feed all qa pairs to LLM as a context and give response to the original query
final_res = (final_template | model).invoke({"context": formatted_string, "question": query})
print(final_res.content)

Meditation is a mental practice that involves focusing one's mind on a particular object, thought, or experience, such as the breath, a mantra, or a visualization. The goal of meditation is to achieve a state of inner peace, clarity, and concentration. Through regular practice, meditation helps individuals develop greater awareness of their thoughts, emotions, and bodily sensations, leading to a more relaxed and centered state of mind. Examples of meditation include deep breathing techniques, mindfulness exercises, and guided meditations. By cultivating a consistent practice, individuals can improve their overall well-being, reduce stress and anxiety, and enhance their ability to concentrate and make decisions.
